# Advanced Cipher Techniques

# 01. Hill Cipher Encryption and Decryption

## **Hill Cipher Overview**
The Hill Cipher is a polygraphic substitution cipher that uses linear algebra for encryption. It operates on blocks of letters and utilizes matrix multiplication for its computations.

### **Encryption Process**
1. **Key Matrix**: Choose a square key matrix (e.g., 2x2 or 3x3) with integer values. Ensure the determinant of the matrix is non-zero and coprime with 26 (for modular arithmetic).
2. **Plaintext Preparation**: Convert plaintext into numerical form (A=0, B=1, ..., Z=25). For a matrix of size NxN, pad the plaintext if its length is not a multiple of N.
3. **Matrix Multiplication**: Multiply the key matrix with the plaintext vector and take modulo 26 for each resulting element.
4. **Ciphertext Conversion**: Convert the resulting numerical values back to letters.

---

### **Encryption Dry Run**
#### Input:
- Plaintext: `ZIAURREHMAN`
- Key Matrix: 
   ```
  K = |  6  24 |
       |  1  13 |
  ```
- Modulo: 26

#### Steps:
1. Convert plaintext into numerical values (ignoring spaces):
   - Z = 25, I = 8, A = 0, U = 20, R = 17, R = 17, E = 4, H = 7, M = 12, A = 0, N = 13
   - Numerical form: `[25, 8, 0, 20, 17, 17, 4, 7, 12, 0, 13]`

2. Group into 2x1 column vectors (since the key matrix is 2x2):
   ```
   [25, 8], [0, 20], [17, 17], [4, 7], [12, 0], [13, 0]
   ```

3. Multiply each vector by the key matrix and take modulo 26:
   - For [25, 8]:
     ```
     C = (K * P) mod 26 = |  6  24 |   | 25 |
                          |  1  13 | * |  8 | mod 26 = |  4 |
                                                       | 25 |
     ```
     Result: `E, Z`
   - Repeat for all vectors.

4. Resulting Ciphertext:
   - Ciphertext: `EZMAQEKRUMGA`

---

### **Python Implementation**

#### **Encryption Function:**

In [1]:
import numpy as np

def hill_cipher_encrypt(plaintext, key):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    plaintext = plaintext.replace(" ", "").upper()
    # Convert plaintext to numerical values
    plaintext_nums = [alphabet.index(char) for char in plaintext]

    # Pad plaintext if necessary
    n = key.shape[0]
    while len(plaintext_nums) % n != 0:
        plaintext_nums.append(alphabet.index('X'))  # Padding with 'X'

    # Reshape plaintext into column vectors
    plaintext_matrix = np.array(plaintext_nums).reshape(-1, n).T

    # Encrypt: Ciphertext = (Key * Plaintext) mod 26
    ciphertext_matrix = (np.dot(key, plaintext_matrix) % 26).T

    # Convert back to letters
    ciphertext = ''.join(alphabet[num] for row in ciphertext_matrix for num in row)
    return ciphertext

# Example
key = np.array([[6, 24], [1, 13]])
plaintext = "ZIAURREHMAN"
ciphertext = hill_cipher_encrypt(plaintext, key)
print("Ciphertext:", ciphertext)

Ciphertext: EZMAQEKRUMGA


C_i = (P_i + K_i) % 26

In [2]:
def vigenere_encrypt(plaintext, keyword):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    keyword_repeated = (keyword.upper() * (len(plaintext) // len(keyword) + 1))[:len(plaintext)]
    
    ciphertext = ''
    for p, k in zip(plaintext.upper(), keyword_repeated):
        shift = (alphabet.index(p) + alphabet.index(k)) % 26
        ciphertext += alphabet[shift]
    
    return ciphertext

# Example
plaintext = "HELLO"
keyword = "KEY"
ciphertext = vigenere_encrypt(plaintext, keyword)
print("Ciphertext:", ciphertext)


Ciphertext: RIJVS


In [3]:
# Example plaintext
plaintext = "HELLO"
rails = 3

# Encryption function
def rail_fence_encrypt(plaintext, rails):
    fence = [['' for _ in range(len(plaintext))] for _ in range(rails)]
    row, col = 0, 0
    direction = 1  # 1 means moving down, -1 means moving up

    for char in plaintext:
        fence[row][col] = char
        col += 1
        row += direction

        if row == 0 or row == rails - 1:
            direction *= -1

    ciphertext = ''.join(''.join(row) for row in fence)
    return ciphertext

# Encrypt the plaintext
ciphertext = rail_fence_encrypt(plaintext, rails)
print(f"Ciphertext: {ciphertext}")


Ciphertext: HOELL


In [4]:
def otp_encrypt(plaintext, key):
    # Ensure that plaintext and key are the same length
    if len(plaintext) != len(key):
        raise ValueError("Plaintext and key must have the same length.")
    
    ciphertext = []
    for p_char, k_char in zip(plaintext, key):
        # Convert characters to numbers (A=0, B=1, ..., Z=25)
        p_val = ord(p_char) - ord('A')
        k_val = ord(k_char) - ord('A')
        
        # Apply the encryption formula
        c_val = (p_val + k_val) % 26
        
        # Convert back to character and append to the ciphertext
        ciphertext.append(chr(c_val + ord('A')))
    
    return ''.join(ciphertext)

# Example plaintext and key
plaintext = "HELLO"
key = "XMCKL"

# Encrypt the plaintext using the One-Time Pad Cipher
ciphertext = otp_encrypt(plaintext, key)
print(f"Ciphertext: {ciphertext}")



Ciphertext: EQNVZ


In [7]:
from Crypto.Cipher import DES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

def des_encrypt(plaintext, key):
    cipher = DES.new(key, DES.MODE_CBC)  # Create DES cipher object
    ciphertext = cipher.encrypt(pad(plaintext.encode(), DES.block_size))  # Pad plaintext and encrypt
    return cipher.iv + ciphertext  # Return IV + ciphertext

# Example usage
key = get_random_bytes(8)  # Generate 8-byte key for DES
plaintext = "Hello DES!"
ciphertext = des_encrypt(plaintext, key)
print(f"Ciphertext (hex): {ciphertext.hex()}")

ModuleNotFoundError: No module named 'Crypto'

In [11]:
def des_decrypt(ciphertext, key):
    iv = ciphertext[:DES.block_size]
    cipher = DES.new(key, DES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext[DES.block_size:]), DES.block_size).decode()
    return plaintext

# Decrypting the previously encrypted message
decrypted_text = des_decrypt(ciphertext, key)
print(f"Decrypted Text: {decrypted_text}")


Decrypted Text: Hello DES!


# 06. Advanced Encryption Standard (AES)

Mathematical Understanding of AES:

The Advanced Encryption Standard (AES) is a symmetric-key algorithm that processes data in blocks of 128 bits using keys of 128, 192, or 256 bits. AES operates using several rounds (10, 12, or 14 rounds depending on the key size) that include substitution (S-box), permutation, and mixing steps.

The AES encryption process consists of four main stages in each round (except the last round):

- SubBytes: Each byte of the data block is substituted using an S-box.
- ShiftRows: Rows of the data are shifted by different offsets.
- MixColumns: The columns of the data matrix are mixed using a matrix multiplication.
- AddRoundKey: The round key is XORed with the data.
- The key expansion algorithm is used to generate the round keys from the original key.

### Encryption Process:

Given a 128-bit block of plaintext and a key (either 128, 192, or 256 bits), AES performs several rounds of substitution, permutation, and mixing operations, depending on the key length.

### Python Encryption and Decryption (using pycryptodome library):

In [12]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes

def aes_encrypt(plaintext, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(plaintext.encode(), AES.block_size))
    return cipher.iv + ciphertext

# Example usage
key = get_random_bytes(16)  # 128-bit key (16 bytes)
plaintext = "Hello AES!"
ciphertext = aes_encrypt(plaintext, key)
print(f"Ciphertext: {ciphertext.hex()}")


Ciphertext: ed01ba1c4277992a022af23ec3f6b2471cb7d9e73861492bc1dc9c4c041214f9


In [13]:
from Crypto.Util.Padding import unpad

def aes_decrypt(ciphertext, key):
    iv = ciphertext[:AES.block_size]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext[AES.block_size:]), AES.block_size).decode()
    return plaintext

# Decrypting the previously encrypted message
decrypted_text = aes_decrypt(ciphertext, key)
print(f"Decrypted Text: {decrypted_text}")


Decrypted Text: Hello AES!
